In [ ]:
import numpy as np
import cv2
import random
import pandas as pd
#Create first set of parents
#First 13 bits are for window width
#Second 13 bits are for window height
def CreateFirstParents(image, NumOfParents):
  Parents = []
  for i in range(NumOfParents):
    width = bin(random.randint(1,image.shape[1]))[2:].zfill(12)
    height = bin(random.randint(1,image.shape[0]))[2:].zfill(12)
    num = width+height
    Parents.append(num)
  return Parents

#Calculate image compression size for a specific window
def windowcompsize(image, x, y, w, h):
  total = w*h
  count = 0
  for j in range(y, y+h):
    for i in range(x, x+w):
      if(image[j,i]==0):
        count+=1
      else:
        count-=1
  if(count == total):
    return 1
  elif(count == total*-1):
    return 2
  else:
    return total+2

#Pad image to have a size divisible by the window size
def padimage(image, w, h):
  width = image.shape[1]
  height = image.shape[0]
  if(w > width):
    image = cv2.copyMakeBorder(image, 0, 0, 0, w-width, cv2.BORDER_CONSTANT, None,  value = 0)
  if(h >  height):
    image = cv2.copyMakeBorder(image, 0, h-height, 0, 0, cv2.BORDER_CONSTANT, None,  value = 0)
  if(w < width):
    maxw = w
    while(maxw < width):
      maxw+=w
    image = cv2.copyMakeBorder(image, 0, 0, 0, maxw-width, cv2.BORDER_CONSTANT, None,  value = 0)
  if(h <  height):
    maxh = h
    while(maxh < height):
      maxh+=h
    image = cv2.copyMakeBorder(image, 0, maxh-height, 0, 0, cv2.BORDER_CONSTANT, None,  value = 0)
  return image

#Finds the new compression size of the image using the parent
def FitnessFunction(image, parent):
  w = int(parent[0:12],2)
  h = int(parent[12:],2)
  curimage = padimage(image, w, h)
  totalsize = 0
  for j in range (0,curimage.shape[0],h):
    for i in range (0,curimage.shape[1],w):
      totalsize+=windowcompsize(curimage, i, j, w, h)
  return totalsize

#Tests all parents to get the best parents
def getbestparents(image, parents, numofparents):
  results = []
  for i in range(0,numparents):
    results.append(FitnessFunction(image, parents[i]))
  resultdataset = pd.DataFrame({'parents':parents, 'results':results})
  resultdataset = resultdataset.sort_values(by=['results'])
  resultdataset = resultdataset[0:int(numofparents/2)]
  return resultdataset

#crossover between parents
def crossover(parents):
  for i in range(0,len(parents),2):
    offspring1 = parents[i][:12] + parents[i+1][12:]
    offspring2 = parents[i+1][:12] +  parents[i][12:]
    parents.append(mutataion(offspring1))
    parents.append(mutataion(offspring2))
  return parents

#change one bit in each number
def mutataion(value):
  index = random.randint(0,23)
  if(value[index]=='0'):
    value = value[:index] + '1' + value[index + 1:]
  else:
    value = value[:index] + '0' + value[index + 1:] 
  return value

In [ ]:
image = cv2.imread('/content/image2.png',0)
binimage = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
#for i in range(0,image.shape[0]):
#  for j in range(0,image.shape[1]):
    #print(image[i,j])
    #image[i,j] = 0

In [ ]:
print(image.shape[0]*image.shape[1])
numparents = 8
parents = []
parents = CreateFirstParents(image,numparents)
for i in range(0,10):
  parentsresults = getbestparents(image, parents, 8)
  parentsresults.insert(2,"Percentage",(parentsresults["results"]/(image.shape[0]*image.shape[1]))*100)
  print(parentsresults)
  parents = parentsresults['parents'].tolist()
  parents = crossover(parents)
for i in range(0,int(numparents/2)):
  x = parents[i][:12]
  y = parents[i][12:]
  x = int(x,2)
  y = int(y,2)
  print(x, y)

640000
                    parents  results  Percentage
4  000001100000000010011010   650604  101.656875
1  000000001001000110111101   685211  107.064219
0  000110010111000100011111   700866  109.510313
2  001001000001000010010100  1024776  160.121250
                    parents  results  Percentage
5  000000000001000010011010   488582   76.340937
0  000001100000000010011010   650604  101.656875
4  000001100000000010111101   671418  104.909063
1  000000001001000110111101   685211  107.064219
                    parents  results  Percentage
0  000000000001000010011010   488582   76.340937
4  000000000001000010011110   499904   78.110000
7  000000001001000011111101   567685   88.700781
1  000001100000000010011010   650604  101.656875
                    parents  results  Percentage
0  000000000001000010011010   488582   76.340937
1  000000000001000010011110   499904   78.110000
6  000000101001000010011010   562185   87.841406
7  000000100000000011111101   566920   88.581250
             